##### Business problem:
- This notebook is for fetching data from Google Search Console monthly metrics.
- **This notebook runs on first day of each month to get data of previous month**

##### Steps: <br>
- Authentication -> service account
- Campaign performance query
- Transform pulled data
- Append in delta

In [0]:
%run ./gsc_api_authentication

  Created wheel for requests-oauth2: filename=requests_oauth2-0.3.0-py3-none-any.whl size=5875 sha256=ad5f524c1d1152c70c0548b6a11e133b243837d9881f439dbe2b714357afab04
  Stored in directory: /root/.cache/pip/wheels/ae/6f/bd/4a73197a7f73128aebb0ab611d95eaa8bacf34acb1f8301288
Successfully built requests-oauth2
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-5cecf66a-971f-4ca0-b92e-458777572bda/bin/python -m pip install --upgrade pip' command.
     |█████████▊                      | 3.3 MB 18.5 MB/

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 177 kB 72.0 MB/s 
     |████████████████████████████████| 96 kB 5.5 MB/s 
     |████████████████████████████████| 120 kB 75.4 MB/s 
     |████████████████████████████████| 218 kB 76.6 MB/s 
     |████████████████████████████████| 409 kB 50.5 MB/s 
     |████████████████████████████████| 155 kB 73.7 MB/s 
     |████████████████████████████████| 77 kB 1.5 MB/s 
  

In [0]:
# payload to get keywords level performance
startdate, enddate = dates_calculation()

# replace with your own sitename
siteUrl = "https://www.yoursitename/"

payload = {
    "startDate": startdate,
    "endDate": enddate,
    "dimensions": ["DATE"],
    "rowLimit": 25000,
    "searchType": "WEB"}

#### Evaluate request content

In [0]:
def transform_data(siteUrl, scopes, service_account_path):
    '''
    This function post request and fetch data from google search console and transform data into normarlized view
    Args: 
        siteUrl: str
        scopes: str
        service_account_path: str
    Returns:
        df: pandads dataframe
    '''
    
    siteUrl = siteUrl
    service_account_path = service_account_path
    
    # create response
    response = post_request(siteUrl, scopes, service_account_path, payload) 
    
    # normalize json to pandas
    df = pd.json_normalize(json.loads(response)["rows"])
    
    
    # add reporting month
    df['updated'] = date.today()
    df['month'] = df['keys'].apply(lambda x: x[0])

    df = df.drop(columns='keys')
    
    # validate if duplicates in month
    if df.duplicated(subset="month").sum() == 0:
        return df
    else:
        return None
        print("There are duplicates in dataset campaign, check on that")

In [0]:
df = transform_data(siteUrl, scopes, service_account_path)

#### Write into DBFS - Ingestion

In [0]:
# your database and table name
yourdatabase = "replacedatabasename"
yourtablename = "replacetablename"

In [0]:
from datetime import date
def ingest_data(df):
    '''
    This function append dataframe into delta lake
    Args:
        df: pandas dataframe
    Returns:
        none
    '''

    # create spark dataframe and save to delta
    sdf = spark.createDataFrame(df)

    sdf.write.format("delta") \
        .mode("append") \
        .saveAsTable(f"{yourdatabase}.{yourtablename}")

In [0]:
ingest_data(df=df)